## 2. Die gesammelten Daten analysieren


Die Daten sind erstmals nur das: Daten. Wir müssen also noch herausfinden, was für Eigenschaften diese überhaupt haben, um sie später verwenden zu können.


Zuerst wieder die Bibliotheken und Konfigurationen...


In [2]:
import json
from collections import defaultdict


mode = 1
# 1 = Nationalratswahlen
# 2 = Ständeratswahlen

filename = "nationalrat_raw.json" if mode == 1 else "ständerat_raw.json"

Hier ist noch eine kleine Hilfsfunktion zum schönen Darstellen einiger Daten


In [3]:
def print_dict(d: dict):
    maxlen_keys = max(map(len, map(str, d.keys())))
    maxlen_values = max(map(len, map(str, d.values())))
    for key in d.keys():
        # abc:.........420
        print(f"{key:.<{maxlen_keys}}{d[key]:.>{maxlen_values+1}}")

Dann folgt das Laden der Daten.


In [5]:
candidates = json.load(open(filename, encoding="utf-8"))

Wie viele Personen kandidieren für jede Partei?


In [ ]:
party_count = defaultdict(int)

for candidate in candidates:
    party = candidate.get("partyAbbreviation")
    if party:
        party_count[party] += 1

# Sortieren nach Anzahl Kandidierende pro Partei
party_count = dict(sorted(party_count.items(), key=lambda item: item[1], reverse=True))
print_dict(party_count)

Die Mitte.........690
GLP...............576
SP................440
FDP...............363
Grüne.............351
SVP...............321
EVP...............239
JM................232
JG................211
JUSO..............206
JFS...............202
JGLP..............183
EDU...............157
JSVP..............156
Ind................84
JEVP...............80
MASS-VOLL!.........59
PdA................55
PP.................45
Aufrecht...........45
engagiert..........39
AL.................31
EAG-LUP............29
SD.................22
LP.................20
MCG................16
Les Libres.........13
SOL................13
Lega................9
PC..................7
PCS.................7
BastA!..............6
JA..................6
DA..................5
Grüne Panther.......5
SE..................5
JBastA..............4
JLB.................4
LDP.................4
LOVB................4
Avanti..............3
Forum Alternativo...3
Parteifrei..........3
Più Donne...........3
DAL.................2
PdA Basel.

Anscheinend gibt es einige Parteien, die nur sehr wenige Mitglieder haben. Beim späteren Bereinigen der Daten müssen wir Kandidierende dieser Parteien leider aussortieren, da eine gewisse Menge an Daten erforderlich ist, um ein neuronales Netzwerk zu trainieren.


Wie sieht's aus mit dem Frauenanteil der Kandidierenden?


In [ ]:
# Ein Dictionary für Frauen und Männer, eins nur für Frauen. Kandidierende ohne Eintrag beim Feld "gender" werden ignoriert
f_and_m = defaultdict(int)
f_only = defaultdict(int)

for candidate in candidates:
    party = candidate.get("partyAbbreviation")
    gender = candidate.get("gender")
    if party:
        if gender == "f":
            f_only[party] += 1
            f_and_m[party] += 1
        if gender == "m":
            f_and_m[party] += 1

# Berechnet den Frauenanteil
female_share = dict(
    [
        (party_name, f"{ratio:.0%}")
        for party_name, ratio in sorted(
            [
                (party_name, women / f_and_m[party_name])
                for party_name, women in f_only.items()
            ],
            key=lambda item: item[1],
            reverse=True,
        )
    ]
)

print_dict(female_share)

Più Donne.........100%
CSP...............100%
neo...............100%
SE.................80%
BastA!.............67%
SOL................67%
JBastA.............67%
JA.................67%
EAG-LUP............66%
DA.................60%
PCS................57%
MCG................56%
AL.................55%
JG.................55%
JEVP...............54%
Grüne..............52%
SP.................51%
JUSO...............50%
LDP................50%
EVP................49%
engagiert..........41%
GLP................40%
PdA................40%
Grüne Panther......40%
JGLP...............39%
Die Mitte..........38%
FDP................38%
JM.................35%
MASS-VOLL!.........34%
Forum Alternativo..33%
Parteifrei.........33%
Aufrecht...........31%
Ind................31%
Les Libres.........31%
JFS................30%
PP.................29%
EDU................29%
JLB................25%
LOVB...............25%
SVP................25%
JSVP...............19%
SD.................18%
PC.................14%
LP.........